In [1]:
import tweepy
from tweepy.auth import OAuthHandler
import pandas as pd
import preprocessor as p
import nltk
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
import re
import spacy

In [26]:
tweets = pd.read_csv('covid_tweets_raw.csv')

In [28]:
nlp = spacy.load('en_core_web_sm')

tweets['tweet'] = tweets['tweet'].apply(lambda x: p.clean(x))

def preprocess_data(data):
    #Removes Numbers
    data = data.astype(str).str.replace('\d+', '')
    lower_text = data.str.lower()
    def lemmatize_text(text):
        doc = nlp(str(text))
        lemmatized = [token.lemma_ for token in doc]
        return lemmatized
    def remove_punctuation(words):
        new_words = []
        for word in words:
            new_word = re.sub(r'[^\w\s]', '', (word))
            if new_word != '':
                new_words.append(new_word)
        return new_words
    words = lower_text.apply(lemmatize_text)
    words = words.apply(remove_punctuation)
    return pd.DataFrame(words)

pre_tweets = preprocess_data(tweets['tweet'])
tweets['tweet'] = pre_tweets
stop_words = set(stopwords.words('english'))
tweets['tweet'] = tweets['tweet'].apply(lambda x: ' '.join([item for item in \
                                    x if item not in stop_words]))

/var/folders/sg/4x_5htrx3y7fm7z77wcz8c_40000gn/T/ipykernel_73994/2376385022.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  data = data.astype(str).str.replace('\d+', '')


In [29]:
tweets

,tweet
0,covid deni website next
1,scotland people hospital covid point pandemict...
2,want pandemic end try away help get close also...
3,world health organization say new covid xe var...
4,covid vaccine unvaccinated aggressively threat...
...,...
1995,billion waste mountain junk ppe buy tory croni...
1996,sweden twitter account covid stat update turn
1997,see exact sht go last year grammys people win
1998,unemployment nation rise covid result gross pe...


In [30]:
tweets.to_csv('covid_tweets.csv')

In [2]:
import json

credentials = json.load(open('credentials.json'))

consumer_key = credentials['consumer_key']
consumer_secret = credentials['consumer_secret']
access_token = credentials['access_token']
access_token_secret = credentials['access_token_secret']

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [3]:
def search_tweets(searchTerm):
    n = 300
    tweets = tweepy.Cursor(api.search_tweets,q=searchTerm,lang="en",tweet_mode="extended").items(n)
    tweets_json=[]
    for tweet in tweets:
        tweets_json.append(tweet._json)
    return tweets_json

In [4]:
covid_tweets = search_tweets('covid')
omicron_tweets = search_tweets('omicron')
world_economy_tweets = search_tweets('world economy')
remittance_tweets = search_tweets('remittance')

In [5]:
def tweets_to_csv(tweets_json, name='covid'):
    tweets = [tweet['full_text'] for tweet in tweets_json]
    tweet_df = pd.DataFrame({'tweet': tweets})
    tweet_df['tweet'] = tweet_df['tweet'].apply(p.clean)
    tweet_df.to_csv(f'{name}_tweets.csv', index=False)

[{'created_at': 'Sat Apr 09 17:02:09 +0000 2022',
  'id': 1512838287819526145,
  'id_str': '1512838287819526145',
  'full_text': 'Alhumdulillah! \nFinally there is no any signal COVID case in pakistan.',
  'truncated': False,
  'display_text_range': [0, 70],
  'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []},
  'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
  'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'in_reply_to_screen_name': None,
  'user': {'id': 1371446693485494276,
   'id_str': '1371446693485494276',
   'name': 'Evil Devil👩🏻\u200d💻',
   'screen_name': 'Evil_Kitten36',
   'location': 'Lahore, Pakistan',
   'description': '2k? ARMY💜',
   'url': None,
   'entities': {'description': {'urls': []}},
   'protected': False,
   'followers_count': 1302,
  

In [8]:
nlp = spacy.load('en_core_web_sm')

def preprocess(tweets, name='covid'):
    tweets = [tweet['full_text'] for tweet in tweets]
    tweets = pd.DataFrame({'tweet': tweets})
    tweets['tweet'] = tweets['tweet'].apply(lambda x: p.clean(x))

    def preprocess_data(data):
        #Removes Numbers
        data = data.astype(str).str.replace('\d+', '')
        lower_text = data.str.lower()
        def lemmatize_text(text):
            doc = nlp(str(text))
            lemmatized = [token.lemma_ for token in doc]
            return lemmatized
        def remove_punctuation(words):
            new_words = []
            for word in words:
                new_word = re.sub(r'[^\w\s]', '', (word))
                if new_word != '':
                    new_words.append(new_word)
            return new_words
        words = lower_text.apply(lemmatize_text)
        words = words.apply(remove_punctuation)
        return pd.DataFrame(words)

    pre_tweets = preprocess_data(tweets['tweet'])
    tweets['tweet'] = pre_tweets
    stop_words = set(stopwords.words('english'))
    tweets['tweet'] = tweets['tweet'].apply(lambda x: ' '.join([item for item in \
                                        x if item not in stop_words]))
    tweets.to_csv(f'{name}_tweets.csv', index=False)
    return tweets

In [9]:
covid_tweets = preprocess(covid_tweets, 'covid')
omicron_tweets = preprocess(omicron_tweets, 'omicron')
world_economy_tweets = preprocess(world_economy_tweets, 'world_economy')
remittance_tweets = preprocess(remittance_tweets, 'remittance')

/var/folders/sg/4x_5htrx3y7fm7z77wcz8c_40000gn/T/ipykernel_46358/738968203.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  data = data.astype(str).str.replace('\d+', '')


In [33]:
# tweets_to_csv(covid_tweets, 'covid')
tweets_to_csv(omicron_tweets, 'omicron')
# tweets_to_csv(world_economy_tweets, 'world_economy')
# tweets_to_csv(remittance_tweets, 'remittance')

In [11]:
covid_tweets['_json']

TypeError: list indices must be integers or slices, not str